<a href="https://colab.research.google.com/github/jialun1221/scRNA-seq/blob/main/Dan_SUpport_Vector_Machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install scanpy
import numpy as np
import pandas as pd
import scanpy as sc
import sys
import random
from sklearn import preprocessing 
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import importlib
required_libraries = ['torch', 'torchvision', 'PIL', 'matplotlib', 
                      'numpy', 'pandas']
for lib in required_libraries:
    if importlib.util.find_spec(lib) is None:
        print("%s unavailable" % lib)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import TensorDataset

In [ ]:
#0. Model methods import
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn import metrics
import sklearn.metrics as metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn import svm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

adata_m1 = sc.read_h5ad("drive/MyDrive/scRNA ML classifier/data_objects_May_2022/DAN_PC_all_genes_qc.h5ad")
adata_m2 = sc.read_h5ad("drive/MyDrive/scRNA ML classifier/data_objects_May_2022/DAN_PC_HVGs_qc.h5ad")
adata_m3 = sc.read_h5ad("drive/MyDrive/scRNA ML classifier/data_objects_May_2022/DAN_no_PC_HVGs_qc.h5ad")

data_list = [adata_m1, adata_m2, adata_m3]

Mounted at /content/drive


In [ ]:
df_astro_SVM = pd.DataFrame(columns = ['Seed', 'dSVM m1 val', 'dSVM m2 val', 'dSVM m3 val','dLR SVM test', 'dSVM m2 test', 'dSVM m3 test'])

In [ ]:
dict_list_astro_SVM_m1 = []
dict_list_astro_SVM_m2 = []
dict_list_astro_SVM_m3 = []

In [ ]:
def SVMTest():
  use_gpu = torch.cuda.is_available()
  device = torch.device("cuda:0" if use_gpu else "cpu") 
  clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  #eval
  test_acc = metrics.accuracy_score(y_test, y_pred)
  print(test_acc)
  return test_acc

In [ ]:
def SVMValidation():
  use_gpu = torch.cuda.is_available()
  device = torch.device("cuda:0" if use_gpu else "cpu") 
  clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
  y_pred_val = clf.predict(X_val)
  #eval
  val_acc = metrics.accuracy_score(y_val, y_pred_val)
  print(val_acc)
  return val_acc

In [ ]:
import copy
i = 0 #method number
seed_list = []

for dataobj in data_list:
  print(i)
  print(dataobj)
  i = i + 1
  adata = dataobj

  #1. Pre-process data
  if 'level_0' not in adata.obs.columns:
    adata.obs = adata.obs.reset_index() #reset the index for X, so the column number starts from 0

  if 'X_pca' in adata.obsm:
    X = adata.obsm['X_pca'].X
  else:
    X = adata.X 
  
  y = adata.obs['disease__ontology_label'].replace({"normal": "0", "Parkinson disease": "1"})

  #2. Train-test split: 60/20/20. Random_state is the seed.Validation used for hyperparameter tuning. 
  #   Parameters for the highest accuracy can be used for test set.
  
  X_train_global, X_test_global, y_train_global, y_test_global = train_test_split(X, y, test_size=0.2, random_state=0) #test is always the same

  #3. run 10 iterations
  for m in range (10):
    X_test = copy.deepcopy(X_test_global)
    y_test = copy.deepcopy(y_test_global)
    
    if len(seed_list) < 10:
      seed = random.randint(0, 9999)
      seed_list.append(seed)
      print("seed: ", seed)
      X_train, X_val, y_train, y_val = train_test_split(X_train_global, y_train_global, test_size=0.25, random_state= seed_list[m])
    else:
      print("seed = ", seed)
      print(seed_list[m])
      X_train, X_val, y_train, y_val = train_test_split(X_train_global, y_train_global, test_size=0.25, random_state= seed_list[m])

    val_acc = SVMValidation()
    test_acc = SVMTest()

    if i == 1: #write in method 1 section
      row_dict_m1 = {'Seed': seed_list[m], 'aSVM m1 val': val_acc, 'aSVM m1 test': test_acc}
      dict_list_astro_SVM_m1.append(row_dict_m1)
    elif i == 2:
      row_dict_m2 = {'Seed': seed_list[m], 'aSVM m2 val': val_acc, 'aSVM m2 test': test_acc}
      dict_list_astro_SVM_m2.append(row_dict_m2)
    elif i == 3:
      row_dict_m3 = {'Seed': seed_list[m], 'aSVM m3 val': val_acc, 'aSVM m3 test': test_acc}
      dict_list_astro_SVM_m3.append(row_dict_m3)
      
print(dict_list_astro_SVM_m1)
print(dict_list_astro_SVM_m2)
print(dict_list_astro_SVM_m3)


0
AnnData object with n_obs × n_vars = 17858 × 32043
    obs: 'index', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'Cell_Subtype', 'Cell_Type', 'disease__ontology_label', 'organ__ontology_label', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'features', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'hvg', 'log1p', 'pca'
    obsm: 'X_pca'
    varm: 'PCs'
seed:  8324
0.9913213885778276
0.9902015677491601
seed:  3079
0.9918812989921613
0.990481522956327
seed:  8926
0.9902015677491601
0.9890817469204927
seed:  2202
0.9902015677491601
0.9902015677491601
seed:  8461
0.9896416573348265
0.9888017917133258
seed:  6304
0.988521836506159
0.9890817469204927
seed:  5855
0.9910414333706606
0.9893617021276596
seed:  6151
0.9913213885778276
0.9907614781634938
seed:  7173
0.9918812989921613
0.9910414333706606
seed:  6352
0.99

In [ ]:
df_astro_m1 = pd.DataFrame.from_dict(dict_list_astro_SVM_m1)
df_astro_m2 = pd.DataFrame.from_dict(dict_list_astro_SVM_m2)
df_astro_m3 = pd.DataFrame.from_dict(dict_list_astro_SVM_m3)
f = pd.concat([df_astro_m1, df_astro_m2,df_astro_m3])
f

,Seed,aSVM m1 val,aSVM m1 test,aSVM m2 val,aSVM m2 test,aSVM m3 val,aSVM m3 test
0,8324,0.991321,0.990202,NaN,NaN,NaN,NaN
1,3079,0.991881,0.990482,NaN,NaN,NaN,NaN
2,8926,0.990202,0.989082,NaN,NaN,NaN,NaN
3,2202,0.990202,0.990202,NaN,NaN,NaN,NaN
4,8461,0.989642,0.988802,NaN,NaN,NaN,NaN
5,6304,0.988522,0.989082,NaN,NaN,NaN,NaN
6,5855,0.991041,0.989362,NaN,NaN,NaN,NaN
7,6151,0.991321,0.990761,NaN,NaN,NaN,NaN
8,7173,0.991881,0.991041,NaN,NaN,NaN,NaN
9,6352,0.991321,0.990482,NaN,NaN,NaN,NaN
